In [1]:
import pandas as pd
import numpy as np
import cupy

def sigmoid(z):
    return cupy.reciprocal(1 + cupy.exp(-z))

In [16]:
U = cupy.array([
    [0.69254207,	0.120757424,	1],
    [0.794932131,	0.506465029,	1],
    [0.087887546,	0.625198524,	1],
    [0.186622288,	0.64901336,	1]
])

V = cupy.array([
    [0.961109006,	0.806823022,	0.906683013,	0.442439555],
    [0.613409475,	0.350356184,	0.527786582,	0.297789197],
    [0.451780345,	0.92009755,	0.848521427,	0.774361515]
])

In [17]:
est_pref = cupy.dot(U,V)

In [18]:
u = 0
I = {0}
J = {2, 3}
K = {1}

spec_alpha_u = 0.5

In [19]:
U_u = U[u, :-1]

r_hat_uI = cupy.average(est_pref[u, sorted(I)]) if len(
                    I) != 0 else cupy.array([0])
r_hat_uJ = cupy.average(est_pref[u, sorted(J)]) if len(
                    J) != 0 else cupy.array([0])
r_hat_uK = cupy.average(est_pref[u, sorted(K)]) if len(
                    K) != 0 else cupy.array([0])

r_hat_uIJ = r_hat_uI - r_hat_uJ
r_hat_uJK = r_hat_uJ - r_hat_uK

V_bar_I = cupy.average(V[:-1, sorted(I)], axis=1) if len(
    I) != 0 else cupy.zeros(
    shape=(2,))
V_bar_J = cupy.average(V[:-1, sorted(J)], axis=1) if len(
    J) != 0 else cupy.zeros(
    shape=(2,))
V_bar_K = cupy.average(V[:-1, sorted(K)], axis=1) if len(
    K) != 0 else cupy.zeros(
    shape=(2,))
# get b_I, b_J, b_K
b_I = cupy.average(V[-1, sorted(I)]) if len(I) != 0 else cupy.array([0])
b_J = cupy.average(V[-1, sorted(J)]) if len(J) != 0 else cupy.array([0])
b_K = cupy.average(V[-1, sorted(K)]) if len(K) != 0 else cupy.array([0])

In [20]:
# Nabla
df_dUu = sigmoid(- r_hat_uIJ / spec_alpha_u) / spec_alpha_u * (V_bar_I - V_bar_J) + \
                         sigmoid(- r_hat_uJK) * (V_bar_J - V_bar_K)
dR_dUu = 2 * 0.1 * U_u

df_dbi = sigmoid(- r_hat_uIJ / spec_alpha_u) / (len(I) * spec_alpha_u)
dR_dbi = 2 * 0.1 * b_I / len(I)
df_dVi = df_dbi * U_u
dR_dVi = 2 * 0.1 * V_bar_I / len(I)

df_dbj = ( - sigmoid(- r_hat_uIJ / spec_alpha_u) / spec_alpha_u + sigmoid(- r_hat_uJK) ) / len(J)
dR_dbj = 2 * 0.1 * b_J / len(J)
df_dVj = df_dbj * U_u
dR_dVj = 2 * 0.1 * V_bar_J / len(J)

df_dbk = - sigmoid(- r_hat_uJK) / len(K)
dR_dbk = 2 * 0.1 * b_K / len(K)
df_dVk = df_dbk * U_u
dR_dVk = 2 * 0.1 * V_bar_K / len(K)

In [21]:
# update
U[u, :-1] += 0.01 * (df_dUu - dR_dUu)

V[:-1, sorted(I)] += 0.01 * (df_dVi - dR_dVi)[:, None]  # trick: transpose here
# update b_i = b_i + 0.01 * (df_dbi - dR_dbi)
V[-1, sorted(I)] += 0.01 * (df_dbi - dR_dbi)

V[:-1, sorted(J)] += 0.01 * (df_dVj - dR_dVj)[:, None]  # trick: transpose here
# update b_j = b_j + 0.01 * (df_dbj - dR_dbj)
V[-1, sorted(J)] += 0.01 * (df_dbj - dR_dbj)

V[:-1, sorted(K)] += 0.01 * (df_dVk - dR_dVk)[:, None]  # trick: transpose here
# update b_k = b_k + 0.01 * (df_dbk - dR_dbk)
V[-1, sorted(K)] += 0.01 * (df_dbk - dR_dbk)

In [23]:
# new est pref
new_est_pref = cupy.dot(U,V)